In [2]:
from tqdm import tqdm
from glob import glob
import numpy as np
import os
from utils.process_data import split_train_val, get_data_properties
# from utils.generate_crops import *
from multiprocessing import Pool

import json

In [6]:
data_dir = '../Data'
project_name = 'H2giga'
if os.path.exists(os.path.join(data_dir,project_name)):
    print("`{}` is choosen as datapath".format(os.path.join(data_dir,project_name)))

`../Data/H2giga` is choosen as datapath


In [7]:
data_properties_dir = get_data_properties(data_dir, project_name, train_val_name=['train', 'val'])


100%|██████████| 24/24 [00:47<00:00,  1.96s/it]


Number of instances per class
 {1: 134, 2: 166, 3: 167, 4: 133, 5: 76}


100%|██████████| 24/24 [00:04<00:00,  4.93it/s]


Foreground weight of the `H2giga` dataset set equal to '{1.0: 7.841997775182617, 2.0: 6.839477258384389, 3.0: 5.68335084423856, 4.0: 9.029515782252188, 5.0: 9.533586911797785}'


100%|██████████| 24/24 [00:45<00:00,  1.90s/it]


Minimum object size of the `H2giga` dataset is equal to 443
Mean object size of the `H2giga` dataset is equal to 8165.266272189349
Maximum object size of the `H2giga` dataset is equal to 33944
Average object size of the `H2giga` dataset along `x` is equal to 102.694
Std. dev object size of the `H2giga` dataset along `x` is equal to 54.010
Average object size of the `H2giga` dataset along `y` is equal to 112.331
Std. dev object size of the `H2giga` dataset along `y` is equal to 60.299


100%|██████████| 24/24 [00:01<00:00, 22.64it/s]

Tile size of the `H2giga` dataset set equal to (1024, 1024)


In [9]:
# n_sigma is used to control the size of crop
n_sigma = 5


def round_up_8(x):
    return (x.astype(int)+7) & (-8)


crops_dir = '../crops/'
data_subsets = ['train', 'val'] 
crop_size =416
# crop_size = np.maximum(round_up_8(data_properties_dir['avg_object_size_y'] + n_sigma*data_properties_dir['stdev_object_size_y']),
# round_up_8(data_properties_dir['avg_object_size_x'] + n_sigma*data_properties_dir['stdev_object_size_x']))
print("Crop size in x and y will be set equal to {}".format(crop_size))



Crop size in x and y will be set equal to 416


In [11]:
import itertools


for data_subset in data_subsets:
    image_dir = os.path.join(data_dir, project_name, data_subset, 'images')
    instance_dir = os.path.join(data_dir, project_name, data_subset,'instances')
    classmap_dir = os.path.join(data_dir, project_name, data_subset,'classmaps')
    hs_dir = os.path.join(data_dir, project_name, data_subset,'hs')
    
    
    
    
    image_names = sorted(glob.glob(os.path.join(image_dir, '*.png'))) 
    instance_names = sorted(glob.glob(os.path.join(instance_dir, '*.png')))
    classmap_names = sorted(glob.glob(os.path.join(classmap_dir, '*.png')))
    hs_names = sorted(glob.glob(os.path.join(hs_dir, '*.cue')))
    header_names = sorted(glob.glob(os.path.join(hs_dir, '*.hdr')))
    crop_size_iter = list(itertools.repeat(crop_size,len(image_names)))
    crop_dir_iter = list(itertools.repeat(os.path.join(crops_dir,'H2giga'),len(image_names)))
    type_iter = list(itertools.repeat(data_subset,len(image_names)))

    
    item = zip(image_names,instance_names,classmap_names,hs_names,header_names,crop_size_iter,crop_dir_iter,type_iter)
    
    with Pool(2) as p:
        max_ = len(image_names)
        with tqdm(total=max_) as pbar:
            for _ in p.imap_unordered(process_with_pool, item):
                pbar.update()
        # shutdown the process pool
        p.close()
        # wait for all issued task to complete
        p.join()

  0%|          | 0/1 [00:00<?, ?it/s]/Users/faruk92/miniconda3/lib/python3.9/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
spectral:WARNING: Unable to parse bad band list (bbl) in ENVI header as integers.
  0%|          | 0/1 [00:00<?, ?it/s]/Users/faruk92/miniconda3/lib/python3.9/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
spectral:WARNING: Unable to parse bad band list (bbl) in ENVI header as integers.
100%|██████████| 1/1 [01:54<00:00, 114.36s/it]
